In [ ]:
from icecube import dataio
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
params = {'legend.fontsize': 16,
          'figure.figsize': (15, 9.3),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 14,
          'ytick.labelsize': 14}
plt.rcParams.update(params)

In [ ]:
D = '/localscratch/weldert/i3files/recoed/'
fits, truths, retros = [], [], []
for f in os.listdir(D):
    if not 'oscNext' in f:
        continue
    file = dataio.I3File(D+f)
    
    while file.more():
        try:
            frame = file.pop_physics()
        except:
            break
        
        # FreeDOM
        fit = frame['FreeDOM_test_best_fit']
        fit.append(np.sum(fit[-2:]))
        fits.append(fit)
        
        # Truth
        neutrino = frame['I3MCTree'][0]
        truth = [neutrino.pos.x, neutrino.pos.y, neutrino.pos.z, neutrino.time, neutrino.dir.azimuth,
                neutrino.dir.zenith] #, neutrino.energy
        cascade_E = 0
        for i in range(1, len(frame['I3MCTree'])):
            if not frame['I3MCTree'][i].is_neutrino and not frame['I3MCTree'][i].is_track:
                cascade_E += frame['I3MCTree'][i].energy
        track_E = 0
        if frame['I3MCTree'][1].is_track:
            track_E += frame['I3MCTree'][1].energy
        truth.extend([cascade_E, track_E, cascade_E+track_E])
        truths.append(truth)
        
        # Retro (max)
        retro = [frame['retro_crs_prefit__x'].values()[1], frame['retro_crs_prefit__y'].values()[1],
                 frame['retro_crs_prefit__z'].values()[1], frame['retro_crs_prefit__time'].values()[1],
                 frame['retro_crs_prefit__azimuth'].values()[1], frame['retro_crs_prefit__zenith'].values()[1],
                 frame['retro_crs_prefit__cascade_energy'].values()[1], frame['retro_crs_prefit__track_energy'].values()[1]]
        retro.append(np.sum(retro[-2:]))
        retros.append(retro)
        
fits, truths, retros = np.array(fits), np.array(truths), np.array(retros)

In [ ]:
ranges = [[-50, 50], [-50, 50], [-50, 50], [-50, 100], [-5, 5], [-2, 2], [-40, 40], [-40, 40], [-50, 50]]
label = ['x', 'y', 'z', 'time', 'azimuth', 'zenith', 'cascade energy', 'track energy', 'energy']

In [ ]:
plt.figure(figsize=(20,15))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.hist(fits[:, i]-truths[:, i], np.linspace(ranges[i][0], ranges[i][1], 100), alpha=0.5, label='FreeDOM')
    plt.hist(retros[:, i]-truths[:, i], np.linspace(ranges[i][0], ranges[i][1], 100), alpha=0.5, label='Retro')
    plt.legend()
    plt.xlabel(label[i])
#plt.savefig('test', bbox_inches='tight')